# Assignment - 1

In [102]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

# Exploratory Data Aanalysis | Data Cleaning

In [103]:
df = pd.read_csv('/content/drive/MyDrive/Machine Learning Datasets/sales_full_data.csv')
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04-07-19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04-12-19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04-12-19 14:38,"669 Spruce St, Los Angeles, CA 90001"


In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


In [105]:
df.isnull().sum()

Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64

In [106]:
df.dropna(axis=0, inplace=True)

In [107]:
df[['Date', 'Time']] = df['Order Date'].str.split(' ', expand=True)
df.drop('Order Date', axis=1, inplace=True)
df.head(2)

,Order ID,Product,Quantity Ordered,Price Each,Purchase Address,Date,Time
0,176558,USB-C Charging Cable,2,11.95,"917 1st St, Dallas, TX 75001",04/19/19,08:46
2,176559,Bose SoundSport Headphones,1,99.99,"682 Chestnut St, Boston, MA 02215",04-07-19,22:30


### While looking into data I observe that some of the rows contains bad data. Let's remove them.

In [108]:
df = df[df['Date'] != 'Order'].reset_index()
df.head(2)

,index,Order ID,Product,Quantity Ordered,Price Each,Purchase Address,Date,Time
0,0,176558,USB-C Charging Cable,2,11.95,"917 1st St, Dallas, TX 75001",04/19/19,08:46
1,2,176559,Bose SoundSport Headphones,1,99.99,"682 Chestnut St, Boston, MA 02215",04-07-19,22:30


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185950 entries, 0 to 185949
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   index             185950 non-null  int64 
 1   Order ID          185950 non-null  object
 2   Product           185950 non-null  object
 3   Quantity Ordered  185950 non-null  object
 4   Price Each        185950 non-null  object
 5   Purchase Address  185950 non-null  object
 6   Date              185950 non-null  object
 7   Time              185950 non-null  object
dtypes: int64(1), object(7)
memory usage: 11.3+ MB


### Converting Date & Time columns from string to Datetime.

In [110]:
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'])
df.head(2)

,index,Order ID,Product,Quantity Ordered,Price Each,Purchase Address,Date,Time
0,0,176558,USB-C Charging Cable,2,11.95,"917 1st St, Dallas, TX 75001",2019-04-19,2021-12-11 08:46:00
1,2,176559,Bose SoundSport Headphones,1,99.99,"682 Chestnut St, Boston, MA 02215",2019-04-07,2021-12-11 22:30:00


### Seperating Year, Month, Day and Time.

In [111]:
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['Time'] = df['Time'].dt.time
df.head(2)

,index,Order ID,Product,Quantity Ordered,Price Each,Purchase Address,Date,Time,year,month,day
0,0,176558,USB-C Charging Cable,2,11.95,"917 1st St, Dallas, TX 75001",2019-04-19,08:46:00,2019,4,19
1,2,176559,Bose SoundSport Headphones,1,99.99,"682 Chestnut St, Boston, MA 02215",2019-04-07,22:30:00,2019,4,7


In [112]:
df.drop(['Date'], axis=1, inplace=True)
df.head(2)

,index,Order ID,Product,Quantity Ordered,Price Each,Purchase Address,Time,year,month,day
0,0,176558,USB-C Charging Cable,2,11.95,"917 1st St, Dallas, TX 75001",08:46:00,2019,4,19
1,2,176559,Bose SoundSport Headphones,1,99.99,"682 Chestnut St, Boston, MA 02215",22:30:00,2019,4,7


In [113]:
df['Price Each'].unique()

array(['11.95', '99.99', '600', '11.99', '1700', '14.95', '389.99',
       '3.84', '150', '2.99', '700', '300', '149.99', '109.99', '999.99',
       '400', '379.99'], dtype=object)

### Price isn't looking good, let's round it to nearest integer.

In [114]:
df['Quantity Ordered'] = df['Quantity Ordered'].astype('int32')
df['Price Each'] = df['Price Each'].astype('float64')
df['Price Each'] = df['Price Each'].apply(lambda x:round(x)).astype('int32')
df.head(2)

,index,Order ID,Product,Quantity Ordered,Price Each,Purchase Address,Time,year,month,day
0,0,176558,USB-C Charging Cable,2,12,"917 1st St, Dallas, TX 75001",08:46:00,2019,4,19
1,2,176559,Bose SoundSport Headphones,1,100,"682 Chestnut St, Boston, MA 02215",22:30:00,2019,4,7


### Creating new feature that contains the total amount of each date and time

In [115]:
df['total'] = df['Quantity Ordered'] * df['Price Each']
df.head(2)

,index,Order ID,Product,Quantity Ordered,Price Each,Purchase Address,Time,year,month,day,total
0,0,176558,USB-C Charging Cable,2,12,"917 1st St, Dallas, TX 75001",08:46:00,2019,4,19,24
1,2,176559,Bose SoundSport Headphones,1,100,"682 Chestnut St, Boston, MA 02215",22:30:00,2019,4,7,100


# Task - 1

### Let's see the Sale of each Month.

In [116]:
x = dict()
for i in range(1,13):
    a = df[df['month'] == i]['total'].sum()
    x[i] = a
print(x)

{1: 1822655, 2: 2202517, 3: 2807722, 4: 3391444, 5: 3153280, 6: 2578373, 7: 2648362, 8: 2244966, 9: 2098053, 10: 3737561, 11: 3200335, 12: 4614482}


In [117]:
data = pd.DataFrame.from_dict(x.items())
data.columns = ['month', 'sale']
data

,month,sale
0,1,1822655
1,2,2202517
2,3,2807722
3,4,3391444
4,5,3153280
5,6,2578373
6,7,2648362
7,8,2244966
8,9,2098053
9,10,3737561


In [118]:
%matplotlib notebook
fig = px.bar(data, x='month', y='sale', width=800, height=500)
fig.show()

In [119]:
%matplotlib notebook
fig = px.pie(data, names='month', values='sale', width=800, height=500, hole=0.5, 
       color_discrete_sequence=px.colors.sequential.Plotly3)
fig.show()

# Max & Min sale Month
* Maximum Sale - December
* Minimum Sale - January

# Task - 2

In [120]:
df['Time'] = df['Time'].astype('str')
df['Time'] = df['Time'].str.split(':')
df['Time'] = df['Time'].apply(lambda x:x[0])
df['Time'] = df['Time'].astype('int32')

In [121]:
df['Time'].unique()

array([ 8, 22, 14,  9, 13,  7, 10, 17, 12, 19, 15, 20, 18,  0, 11, 23, 21,
        4, 16,  5,  2,  1,  6,  3], dtype=int32)

In [122]:
x = dict()
for i in range(0,24):
    a = df[df['Time'] == i]['total'].sum()
    x[i] = a

data = pd.DataFrame.from_dict(x.items())
data.columns = ['Time', 'Sale']
data

,Time,Sale
0,0,713874
1,1,460959
2,2,234902
3,3,145792
4,4,162691
5,5,230735
6,6,448217
7,7,745025
8,8,1192603
9,9,1639386


In [123]:
%matplotlib tk
fig = px.bar(data, x='Time', y='Sale', width=800, height=400)
fig.show()

## Clearly For increasing sales, I'll put Advertisements Between - 11:00 AM to 9:00 PM.

# Task - 3

In [124]:
df['Product'].value_counts()

USB-C Charging Cable          21903
Lightning Charging Cable      21658
AAA Batteries (4-pack)        20641
AA Batteries (4-pack)         20577
Wired Headphones              18882
Apple Airpods Headphones      15549
Bose SoundSport Headphones    13325
27in FHD Monitor               7507
iPhone                         6842
27in 4K Gaming Monitor         6230
34in Ultrawide Monitor         6181
Google Phone                   5525
Flatscreen TV                  4800
Macbook Pro Laptop             4724
ThinkPad Laptop                4128
20in Monitor                   4101
Vareebadd Phone                2065
LG Washing Machine              666
LG Dryer                        646
Name: Product, dtype: int64

In [127]:
%matplotlib tk
temp = df.groupby(by=['Product']).size().reset_index(name="counts")
fig = px.bar(temp, x='Product', y='counts')
fig.show()

## USB-C Charging Cable, Lightning Charging Cable were sold the most.